<a href="https://colab.research.google.com/github/yimuzy/Is20f/blob/master/Fourier_Transform_Heston_BSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BSM Model



Black-Scholes Model for Call Option Pricing
$$
C = SN(d_1)-Ke^{-rT}N(d_2) \\
d_1 = \frac{1}{\sigma\sqrt{T-t}}[ln(\frac{S_t}{K})+(r+\frac{\sigma^2}{2})(T-t)] \\
d_2 = d_1 - \sigma\sqrt{T-t}
$$

In [ ]:
import scipy.integrate as int
import numpy as np
import scipy as scp
import scipy.stats as ss
import matplotlib.pyplot as plt
import scipy.special as scps
from statsmodels.graphics.gofplots import qqplot
from scipy.linalg import cholesky
from functools import partial

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
r= .0475; sigma=  .2; otype = 1; T = 1;

In [ ]:
K = []
s0 = []
for i in range (10000):
  K.append(110+i)
  s0.append(100+i)
 

In [ ]:
%%time
#Calculate I_1
d1 = []
I1 = []
for i in range (10000):
  d1.append((np.log(s0[i]/K[i])+(r+(sigma**2)/2)*T)/(sigma*np.sqrt(T)))
  I1.append((1/2)+(1/np.pi)*int.quad(lambda t: (np.sin(d1[i]*t)/t)*np.exp((-1)*t**2/2),0,np.inf)[0])
#Calculate I_2
d2 = []
I2 = []
for i in range (10000):
  d2.append((np.log(s0[i]/K[i])+(r-(sigma**2)/2)*T)/(sigma*np.sqrt(T)))
  I2.append((1/2)+(1/np.pi)*int.quad(lambda t: (np.sin(d2[i]*t)/t)*np.exp((-1)*t**2/2),0,np.inf)[0])
#Get Price
C = []
for i in range(10000):
  C.append((s0[i]*I1[i])-K[i]*np.exp(-r*T)*I2[i])


CPU times: user 7.41 s, sys: 13.3 ms, total: 7.42 s
Wall time: 7.43 s


Fourier Transform Call Option Pricing

$$
\Psi_T(v) = \int_{-\infty}^{\infty}e^{ivk}c_T(k)dk \\
c_T(k) = exp(\alpha k)C_T(k) \\
C_T(k) = \frac{exp(-\alpha k)}{\pi}\int_{0}^{\infty}e^{-ivk}\psi(v)dv
$$

$$
C_T(k) = \frac{exp(-\alpha k)}{\pi}\sum^{N}_{j=1}e^{-iv_jk}\psi_T(v_j)\eta
$$

k = ln(K) </br>
$
\lambda\eta = \frac{2\pi}{N}
$</br>
$b = \frac{1}{2}N\lambda$</br>


In [ ]:
def BSM_characteristic_function(v, x0, T, r, sigma):
    cf_value = np.exp(((x0 / T + r - 0.5 * sigma ** 2) * 1j * v
                - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value
def BSM_call_characteristic_function(v,alpha, x0, T, r, sigma):
    res=np.exp(-r*T)/((alpha+1j*v)*(alpha+1j*v+1))\
        *BSM_characteristic_function((v-(alpha+1)*1j), x0, T, r, sigma)
    return res
    
def SimpsonW(N,eta):
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = eta*(3 + (-1) ** j - delt) / 3
    return SimpsonW

In [ ]:
def Fourier_Transform_Integral(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N=2**10
    B=100
    eta=B/N
    W=SimpsonW(N,eta)
    alpha=1.5
    integral=0
    
    for j in range(N):
        v_j=j*eta
        segment=np.exp(-1j*v_j*k)*BSM_call_characteristic_function(v_j,alpha, x0, T, r, sigma)*W[j]            
        integral+=segment.real

        
    return integral*np.exp(-alpha*k)/np.pi

In [ ]:
%%time
C_FT = []
for i in range(10000):
  S0_FT = s0[i]
  K_FT = K[i] 
  C.append(Fourier_Transform_Integral(S0_FT, K_FT, T, r, sigma))


CPU times: user 2min 14s, sys: 13.9 ms, total: 2min 14s
Wall time: 2min 14s


Fast Fourier Transform

$$
C_T(k_u) = \frac{exp(-\alpha k)}{\pi}\sum^{N}_{j=1}e^{-i\lambda\eta(j-1)(u-1)}e^{ibv_j}\psi_T(v_j)\eta
$$

In [ ]:
def FFT(S0, K, T, r, sigma):
    k = np.log(K)
    x0 = np.log(S0)
    N =2**10
    alpha=1.5
    
    eta=0.15
    lambda_ = 2 * np.pi / (N *eta)
    beta=x0-lambda_*N/2
    km=np.asarray([beta+i*lambda_ for i in range(N)])
    W=SimpsonW(N,eta)
    v=np.asarray([i*eta for i in range(N)])
    Psi=np.asarray([BSM_call_characteristic_function(vj,alpha, x0, T, r, sigma)  for vj in v])
    FFTFunc=Psi*np.exp(-1j*beta*v)*W
    y=np.fft.fft(FFTFunc).real
    cT=np.exp(-alpha*km)*y/np.pi
    
    return np.exp(km),cT

In [ ]:
%%time
C_FFT = []
for i in range (10000):
  S0_FFT = s0[i]
  K_FFT = K[i]
  k_fft,c_fft = FFT(S0_FFT, K_FFT, T, r, sigma)
  C_FFT.append(np.interp(K_FFT, k_fft, c_fft))

CPU times: user 2min 27s, sys: 137 ms, total: 2min 27s
Wall time: 2min 27s
